<a href="https://colab.research.google.com/github/LucasMatuszewski/Python-colab-notebooks/blob/main/CrewAI_Autonomus_Agents_Team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crew AI - Multi Agent
https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/



## Install dependencies

In [2]:
!pip install crewai
!pip install 'crewai[tools]'
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 629.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

## Choose Platform, Model and get API Key

In [34]:
import os
from google.colab import userdata
secretName = "OpenAIkey" # @param ["MistralKey", "OpenAIkey"] {type:"string"}
my_api_key = userdata.get(secretName)
os.environ['OPENAI_API_KEY'] = my_api_key

if secretName == "MistralKey":
  os.environ['OPENAI_API_BASE']="https://api.mistral.ai/v1"
  os.environ['OPENAI_MODEL_NAME']="mistral-small"
else:
  os.environ['OPENAI_API_BASE']="https://api.openai.com/v1"
  os.environ["OPENAI_MODEL_NAME"]="gpt-4-0125-preview"

## Set Agents and Topic

In [35]:
from crewai import Agent
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

# Topic for the crew run
topic = 'IT training & Business training services'
default_max_rpm=10

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal=f'Uncover groundbreaking technologies in {topic}',
  verbose=True,
  max_iter=4,
  max_rpm=default_max_rpm,
  memory=True,
  backstory="""Driven by curiosity, you're at the forefront of innovation,
    eager to explore and share knowledge that could change the world.""",
  tools=[search_tool],
  allow_delegation=True
)

analyst = Agent(
  role='Data Analyst',
  goal=f'Analyze research findings and data about {topic}',
  backstory="""A meticulous analyst with a knack for uncovering patterns
    and finding interesting and insightfull data.""",
  verbose=True,
  max_iter=4,
  max_rpm=default_max_rpm,
  memory=True,
  tools=[search_tool],
  allow_delegation=True
)

writer = Agent(
  role='Writer',
  goal=f'Narrate compelling tech stories about {topic}',
  verbose=False,
  max_iter=3,
  max_rpm=default_max_rpm,
  memory=True,
  backstory="""With a flair for simplifying complex topics,
   you craft engaging narratives that captivate and educate,
   bringing new discoveries to light in an accessible manner.""",
  tools=[search_tool],
  allow_delegation=True
)

marketer = Agent(
  role='Marketing expert',
  goal=f'Optimize all our efforts for parctical implementations, to increase sales and marketing impact of {topic}',
  verbose=True,
  max_iter=3,
  max_rpm=default_max_rpm,
  memory=True,
  backstory="""With a passion to psychology of sales, designing new product and creating marketing strategies,
    you are always looking for ways to transform ideas into products and real actionable plans.""",
  # tools=[search_tool],
  allow_delegation=True
)

proofReader = Agent(
  role='Proofreader',
  goal='Check facts, find wrong data, correct grammar, speling and markdown formatting of the final article',
  backstory='Skilled proofreader with passion to finding errors, mistakes, typos, false information and unverified data.',
  verbose=False,
  max_iter=3,
  max_rpm=default_max_rpm,
  memory=True,
  tools=[search_tool],
  allow_delegation=False
)

## Set Tasks for Agents

In [36]:
from crewai import Task

# Research task
research_task = Task(
  description=f"""Identify the next big trend in {topic}.
  Focus on identifying pros and cons and the overall narrative.
  Your final report should clearly articulate the key points,
  its market opportunities, and potential risks.""",
  expected_output=f'A comprehensive list of 6 bullet points with latest trends in {topic}',
  tools=[search_tool],
  agent=researcher,
  async_execution=True,
)

# Writing task with language model configuration
analyst_task = Task(
  description=f"""Identify the most interesting data and facts about {topic}.
  Focus on identifying real data from researches, statistics, raports, and create insights.
  Your final report should clearly articulate the key points, most interesting and surprising data,
  its market opportunities, and potential risks.""",
  expected_output=f'A comprehensive list of 5 bullet points with latest data and reaserch in {topic}',
  tools=[search_tool],
  agent=analyst,
  async_execution=True,
  # output_file='new-blog-post.md'  # Example of output customization
)

# Writing task with language model configuration
write_task = Task(
  description=f"""Compose an insightful article on {topic}.
  Focus on the latest trends, facts, data and how it's impacting the industry.
  This article should be easy to understand, engaging, and positive.""",
  expected_output=f'A 10 paragraph article on {topic} formated as markdown.',
  # tools=[search_tool],
  agent=writer,
  async_execution=False,
  context=[research_task, analyst_task],
  # output_file='new-blog-post.md'  # Example of output customization
)

# Marketing task with language model configuration
marketing_task = Task(
  # description=f"""Create an marketing strategy and select best ideas how to use {topic} and how to build products and services for IT training company based on Ai.
  # Propose ideas how to use AI in optimization of processes and in marketing.
  # Create an action plan with ready to start points and instructions.""",
  # expected_output=f'A 15 paragraph strategy on {topic} fromated as markdown.',
  description=f"""Optimize the article on {topic} to make it good as a sales and marketing tool.
  This article should make the readers want to buy our services and leave them with feeling that we are interesting company.
  But it should be done without any direct sales or call to action. Very subtle.
  It should be done in a smart way with the use of your knowledge about the psychology of marketing. For example, you can use AIDA model and anchoring.""",
  expected_output=f'A 10 paragraph article on {topic} fromated as markdown.',
  tools=[search_tool],
  agent=marketer,
  async_execution=False,
  context=[write_task],
  # output_file='new-blog-post.md'  # Example of output customization
)

# Proofreading task with language model configuration
proofreading_task = Task(
  description=f"""Check the article on {topic} to find all possible typos, errors, mistakes, and wrong formatting in Markdown.
  Check all data and facts if they are correct.""",
  expected_output=f'A 10 paragraph article on {topic} fromated as markdown.',
  tools=[search_tool],
  agent=marketer,
  async_execution=False,
  context=[marketing_task],
  output_file='new-blog-post.md'  # Example of output customization
)

## Form the Crew

In [37]:
from langchain_openai import ChatOpenAI
from crewai import Crew, Process

# Forming the tech-focused crew with enhanced configurations
crew = Crew(
  agents=[researcher, analyst, writer, marketer, proofReader],
  tasks=[research_task, analyst_task, write_task, marketing_task, proofreading_task],
  process=Process.sequential, # Optional: Sequential task execution is default
  max_rpm=30,
  full_output=True,
  # language="Polish" # ???

  # manager_llm=ChatOpenAI(temperature=0, model="gpt-4"),  # Defines the manager's decision-making engine
  # process=Process.hierarchical  # Specifies the hierarchical management approach
)

## Kick It Off

In [38]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff()
print(result)



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Use Tool: duckduckgo_search
Input: latest research on IT training services 2023Use Tool: duckduckgo_search
Input: latest trends in IT training & Business training services 2023 

McKinsey Technology Trends Outlook 2023. (81 pages) After a tumultuous 2022 for technology investment and talent, the first half of 2023 has seen a resurgence of enthusiasm about technology's potential to catalyze progress in business and society. Generative AI deserves much of the credit for ushering in this revival, but it stands as just ... Shehan Jayawardene July 10, 2023. T he IT industry is constantly evolving, and so too is the way that IT professionals are trained. In recent years, we have seen a number of new trends emerge in IT training, including the rise of online training, the growing importance of soft skills, the use of gamification, and the focus on real-world skills. Dec 5, 2023. On-the-job training remains